In [3]:
import pandas as pd
import networkx as nx
import subprocess
import matplotlib.pyplot as plt
import gensim
import os

from networkx.drawing.nx_agraph import graphviz_layout
from chinese_whispers import chinese_whispers, aggregate_clusters
from gensim.models.poincare import PoincareModel
from nltk.corpus import wordnet as wn

### Construct the Networkx graph
From a csv file

In [ ]:
def display_taxonomy(graph):
    """ Display the taxonomy in a hierarchical layout """
    pos = graphviz_layout(graph, prog='dot', args="-Grankdir=LR")
    plt.figure(3,figsize=(48,144))
    nx.draw(graph, pos, with_labels=True, arrows=True)
    plt.show()

In [ ]:
input_path = 'taxi_output/simple_full/science/science_en.csv-relations.csv-taxo-knn1.csv'

In [ ]:
# Read the taxonomy as a dataframe
df = pd.read_csv(
    input_path,
    sep='\t',
    header=None,
    names=['hyponym', 'hypernym'],
    usecols=[1,2],
)

In [ ]:
# Construct the networkx graph
G = nx.DiGraph()
for rel in zip(list(df['hypernym']), list(df['hyponym'])):
    
    rel_0 = rel[0]
    rel_1 = rel[1]
    
    # Simplify the compound words by replacing the whitespaces with underscores
    if ' ' in rel[0]:
        rel_0 = '_'.join(rel[0].split())
    if ' ' in rel[1]:
        rel_1 = '_'.join(rel[1].split())
    G.add_edge(rel_0, rel_1)

In [ ]:
list(nx.isolates(G))

## Load Word Vectors

In [1]:
def load_vectors():
    """ Load word vectors. """

    embedding_dir = '/home/5aly/taxi/distributed_semantics/embeddings/'

    poincare_model = model = PoincareModel.load(embedding_dir + 'embeddings_poincare_wordnet')  # parent-cluster relationship
    own_model = gensim.models.KeyedVectors.load(embedding_dir + 'own_embeddings_w2v')  # family-cluster relationship

    return poincare_model, own_model

In [4]:
poincare_w2v, own_w2v = load_vectors()

In [5]:
own_w2v.most_similar('bread')

/home/acharya/anaconda3/envs/tax3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/home/acharya/anaconda3/envs/tax3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('flatbread', 0.7123582363128662),
 ('breads', 0.7097251415252686),
 ('unleavened', 0.7093826532363892),
 ('gruel', 0.700811505317688),
 ('dough', 0.6893404722213745),
 ('flour', 0.6792106628417969),
 ('matzo', 0.6730862855911255),
 ('loaves', 0.6616495847702026),
 ('curd', 0.6615822315216064),
 ('cheese', 0.6587767601013184)]

# Improving Taxonomy with Distributional Semantics

Create a networkx graph for each node containing only its children. Draw edges among the children based on the similarity with one another using word vectors.

In [ ]:
def create_children_clusters(own_model, graph):
    """ This function returns a dictionary where corresponding to each key(node) is a graph of its children """
    
    clustered_graph = {}
    for node in graph.nodes():
        clustered_graph[node] = nx.Graph()
        successors = [s.lower() for s in graph.successors(node)]

        for successor in successors:
            try:
                for word, _ in own_model.most_similar(successor, topn=100):
                    if word.lower() in successors:
                        clustered_graph[node].add_edge(successor, word.lower())
            except KeyError:  # If the word in not in vocabulary, check using the substring based method
                successor_terms = successor.split('_')
                if node in successor_terms:
                    clustered_graph[node].add_node(successor)
    
    return clustered_graph

In [ ]:
GC = create_children_clusters(own_w2v, G)

In [ ]:
posI = graphviz_layout(GC['engineering'])
# plt.figure(2, figsize=(20, 20))
nx.draw(GC['engineering'], posI, with_labels=True, arrows=True)
plt.show()

## Implementing Chinese Whispers Algorithm

### Removal of smaller clusters
- For every node, cluster its children.
- Keep only the biggest cluster and detach the rest from the graph.  
- Store the removed clusters in a list.

In [ ]:
def remove_clusters(own_model, nx_graph):
    """ Removes the less related and small clusters from the graph """

    print('Removing small clusters..')
    g_clustered = create_children_clusters(own_model, nx_graph)
    removed_clusters = []

    nodes, clusters, size_ratio = [], [], []
    for node, graph in g_clustered.items():
        gc = chinese_whispers(graph, weighting='top', iterations=60)
        try:  # Get the length of the largest cluster
            max_cluster_size = len(max(aggregate_clusters(gc).values(), key=len))
        except ValueError:
            continue
        
        # Calculate the size ratio of all the clusters which are smaller than the largest
        for _, cluster in aggregate_clusters(gc).items():
            if len(cluster) < max_cluster_size:
                nodes.append(node)
                clusters.append(cluster)
                size_ratio.append(len(cluster) / max_cluster_size)
    
    # Sort the small clusters according to their size_ratio
    sorted_node_clusters = [(node, cluster) for _, cluster, node in sorted(zip(size_ratio, clusters, nodes))]
    if len(sorted_node_clusters) > 10:
        sorted_node_clusters = sorted_node_clusters[:10]

    for node, cluster in sorted_node_clusters:  # detach only the smallest 10 clusters in the entire taxonomy
        removed_clusters.append(cluster)
        for item in cluster:
            nx_graph.remove_edge(node, item)

    return nx_graph, removed_clusters

In [ ]:
G_improved = G.copy()
G_improved, removed_clusters = remove_clusters(own_w2v, G_improved)

In [ ]:
len(removed_clusters)

### Adding back the removed clusters
- Loop through all the removed clusters.
- For each removed cluster, find out the cluster in the graph that has the maximum similarity with it.

Similarity between two clusters is computed by calculating the average of the pairwise similarity of the elements of both the clusters i.e. NxM

In [ ]:
GC_detached = create_children_clusters(own_w2v, G_improved)

In [ ]:
def calculate_similarity(poincare_model, own_model, parent, family, cluster, exclude_parent, exclude_family):
    
    # Similarity between the parent and a cluster
    parent_similarity = 0
    if not exclude_parent:
        parent_similarities = []
        for item in cluster:
            max_similarity = 0
            item_senses = [i_sense.name() for i_sense in wn.synsets(item) if item in i_sense.name()]
            parent_senses = [p_sense.name() for p_sense in wn.synsets(parent) if parent in p_sense.name()]
            for parent_sense in parent_senses:
                for item_sense in item_senses:
                    try:
                        similarity = poincare_model.kv.similarity(parent_sense, item_sense)
                        if similarity > max_similarity:
                            max_similarity = similarity
                    except KeyError as e:
                        if parent_sense.name() in str(e):
                            break
                        else:
                            continue
            if max_similarity != 0:
                parent_similarities.append(max_similarity)
        if len(parent_similarities) > 0:  # Happens when the cluster has only one item which is not in vocabulary
            parent_similarity = sum(parent_similarities) / len(parent_similarities)
    
    # Similarity between a family and a cluster
    family_similarity = 0
    if not exclude_family:
        family_similarities = []
        for f_item in family:
            for c_item in cluster:
                try:
                    family_similarities.append(own_model.similarity(f_item, c_item))
                except KeyError as e:  # skip the terms not in vocabulary
                    if f_item in str(e):
                        break
                    else:
                        continue
        if len(family_similarities) > 0:
            family_similarity = sum(family_similarities) / len(family_similarities)
    
    # Final score is the average of both the similarities
    return (parent_similarity + family_similarity) / 2

In [ ]:
for cluster in removed_clusters:
    max_score = 0
    max_score_node = ''
    for node, graph in GC_detached.items():
        gc = chinese_whispers(graph, weighting='top', iterations=60)
        for label, family in aggregate_clusters(gc).items():
            score = calculate_similarity(poincare_w2v, own_w2v, node, family, cluster, False, False)
            if score > max_score:
                max_score = score
                max_score_node = node
    for item in cluster:
        G_improved.add_edge(max_score_node, item)

### Tuning the nodes and the edges

In [ ]:
def tune_result(g_improved):
    """ Filter the results i.e. remove all the isolated nodes and nodes with blank labels """

    print('\nTuning the result...')

    if '' in g_improved.nodes():
        g_improved.remove_node('')

    hypernyms = {x[0] for x in g_improved.edges()}
    isolated_nodes = list(nx.isolates(g_improved))
    for isolated_node in isolated_nodes:
        terms = isolated_node.split('_')
        if terms[-1] in hypernyms:
            g_improved.add_edge(terms[-1], isolated_node)
        elif terms[0] in hypernyms:
            g_improved.add_edge(terms[0], isolated_node)
        else:
            g_improved.remove_node(isolated_node)

    return g_improved

In [ ]:
tune_result(G_improved)
print('Tuned.')

In [ ]:
list(nx.isolates(G_improved))

# Scores

In [ ]:
def get_line_count(file_name):
    """ Counts the number of lines in a file using bash """

    return int(subprocess.check_output(
        "wc -l {file_name} | grep -o -E '^[0-9]+'".format(file_name=file_name), shell=True
    ).decode('utf-8').split('\n')[0])

In [ ]:
def graph_pruning(path, output_dir, domain):

    cycle_removing_tool = "graph_pruning/graph_pruning.py"
    cycle_removing_method = "tarjan"
    cleaning_tool = "graph_pruning/cleaning.py"

    input_file = os.path.basename(path)
    file_pruned_out = input_file + '-pruned.csv'
    file_cleaned_out = file_pruned_out + '-cleaned.csv'

    print('======================================================================================================================')
    cycle_removing_cmd = 'python {cycle_removing_tool} {input_path} {output_dir}/{file_pruned_out} {cycle_removing_method} | tee /dev/tty'.format(
        cycle_removing_tool=cycle_removing_tool,
        input_path=path,
        output_dir=output_dir,
        file_pruned_out=file_pruned_out,
        cycle_removing_method=cycle_removing_method
    )
    print('Cycle removing:', cycle_removing_cmd, sep='\n')
    subprocess.check_output(cycle_removing_cmd, shell=True)
    print('Cycle removing finished. Written to: {output_dir}/{file_pruned_out}\n'.format(
        output_dir=output_dir, file_pruned_out=file_pruned_out
    ))

    print('======================================================================================================================')
    cleaning_cmd = 'python {cleaning_tool} {output_dir}/{file_pruned_out} {output_dir}/{file_cleaned_out} {domain}'.format(
        cleaning_tool=cleaning_tool,
        output_dir=output_dir,
        file_pruned_out=file_pruned_out,
        file_cleaned_out=file_cleaned_out,
        domain=domain
    )
    print('Cleaning:', cleaning_cmd, sep='\n')
    subprocess.check_output(cleaning_cmd, shell=True)
    print('Finished cleaning. Write output to: {output_dir}/{file_cleaned_out}\n'.format(
        output_dir=output_dir, file_cleaned_out=file_cleaned_out
    ))

    return os.path.join(output_dir, file_cleaned_out)

In [ ]:
def calculate_f1_score(system_generated_taxo, output_dir, domain):
    """ Calculate the F1 score of the re-generated taxonomies """

    eval_tool = 'eval/taxi_eval_archive/TExEval.jar'
    eval_gold_standard = 'eval/taxi_eval_archive/gold_standard/{domain}.taxo'.format(domain=domain)
    eval_root = domain
    eval_jvm = '-Xmx9000m'
    eval_tool_result = os.path.join(output_dir, os.path.basename(system_generated_taxo)) + '-evalresult.txt'

    # Running the tool
    print('======================================================================================================================')
    tool_command = """java {eval_jvm} -jar {eval_tool} {system_generated_taxo} {eval_gold_standard} {eval_root} {eval_tool_result}""".format(
        eval_jvm=eval_jvm,
        eval_tool=eval_tool,
        system_generated_taxo=system_generated_taxo,
        eval_gold_standard=eval_gold_standard,
        eval_root=eval_root,
        eval_tool_result=eval_tool_result
    )
    print('Running eval-tool:', tool_command, sep='\n')
    subprocess.check_output(tool_command, shell=True)
    print('Result of eval-tool written to:', eval_tool_result)

    # Calculating Precision, F1 score and F&M Measure
    scores = {}
    l_gold = get_line_count(eval_gold_standard)
    l_input = get_line_count(system_generated_taxo)

    scores['recall'] = float(subprocess.check_output(
        r"tail -n 1 {eval_tool_result} | grep -o -E '[0-9]+[\.]?[0-9]*'".format(
            eval_tool_result=eval_tool_result
        ), shell=True
    ).decode('utf-8').split('\n')[0])
    scores['precision'] = scores['recall'] * l_gold / l_input

    scores['f1'] = 2 * scores['recall'] * scores['precision'] / (scores['recall'] + scores['precision'])
    scores['f_m'] = float(subprocess.check_output(
        r"cat {eval_tool_result} | grep -o -E 'Cumulative Measure.*' | grep -o -E '0\.[0-9]+'".format(
            eval_tool_result=eval_tool_result
        ), shell=True
    ).decode('utf-8').split('\n')[0])

    # Display results
    print('\nPrecision:', scores['precision'])
    print('Recall:', scores['recall'])
    print('F1:', scores['f1'])
    print('F&M:', scores['f_m'])

    return scores

## Save the result

In [ ]:
def save_result(result, path, mode):
    print('\nSaving the result...')
    df_improved = pd.DataFrame(list(result.edges()), columns=['hypernym', 'hyponym'])
    df_improved = df_improved[df_improved.columns.tolist()[::-1]]

    # Replace the underscores with blanks
    df_improved['hyponym'] = df_improved['hyponym'].apply(lambda x: x.replace('_', ' '))
    df_improved['hypernym'] = df_improved['hypernym'].apply(lambda x: x.replace('_', ' '))

    # Store the result
    output_path = os.path.join(
        'taxi_output', 'distributional_semantics',
        os.path.basename(path) + '-' + mode + os.path.splitext(path)[-1]
    )
    df_improved.to_csv(output_path, sep='\t', header=False)
    print('Output saved at:', output_path)

    return output_path

In [ ]:
output_path = save_result(G_improved, input_path, 'reattach')

In [ ]:
pruned_output = graph_pruning(output_path, 'out', 'science')

In [ ]:
score = calculate_f1_score(pruned_output, 'out', 'science')

## Results visualization

### Clusters

In [ ]:
def visualize_clusters(graph):
    """ Clusterize the nodes of a particular domain in a given graph """
    graph_cluster = chinese_whispers(graph, weighting='top', iterations=60)
    
    # Visualize the clustering of graph_cluster using NetworkX (requires matplotlib)
    colors = [1. / graph_cluster.node[node]['label'] for node in graph_cluster.nodes()]
    fig = plt.gcf()
    fig.set_size_inches(20, 20)
    nx.draw_networkx(graph_cluster, cmap=plt.get_cmap('jet'), node_color=colors, font_color='black')
    plt.show()

In [ ]:
GC_improved = create_children_clusters(own_w2v, G_improved)

In [ ]:
domain = 'mechanical_engineering'

In [ ]:
# Original clusters
visualize_clusters(GC[domain])

In [ ]:
# Clusters after detaching
visualize_clusters(GC_detached[domain])

In [ ]:
# Clusters after detaching and re-attaching the clusters
visualize_clusters(GC_improved[domain])

### Taxonomy

In [ ]:
# View the original taxonomy
display_taxonomy(G)

In [ ]:
# View the modified taxonomy
display_taxonomy(G_improved)

In [ ]:
len(list(G.nodes()))

In [ ]:
len(list(G_improved.nodes()))